In [35]:
import os
import sys

current_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

from apikey import apikey, langchain_apikey

os.environ['OPENAI_API_KEY'] = apikey
os.environ['LANGCHAIN_API_KEY'] = langchain_apikey
os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [36]:
# 用langchain 做prompt 比较方便

# LCEL 基础

LangChain Expression Language (LCEL)


## LCEL - prompt 细节

In [37]:
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")


In [38]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [45]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [46]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

## LCEL - model 细节

In [41]:
# 这种方式只能用langchain包装的openai，不能直接用openai的接口

In [43]:
from langchain_openai.llms import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct")
message = model.invoke(prompt_value)
# cc：message 直接就是llm返回后的结果
message

'\n\nRobot: Why did the ice cream truck break down? Because it had a rocky road!'

In [48]:
type(message) # 直接是str 类型了，并不是 ChatMessage 类型

str

## LCEL - parser 细节

In [13]:
# 对输出进行解析

In [18]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

output_parser.invoke(message)



"\n\nRobot: Why couldn't the bicycle stand up by itself? Because it was two-tired."

In [19]:
input = {"topic": "ice cream"}

prompt.invoke(input)

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [21]:
(prompt | model).invoke(input)

'\n\nRobot: Why did the ice cream go to therapy? Because it was feeling a little melon-collie.'

## LCEL - chain ，将以上每个部分合在一起形成chain

In [44]:
# 以上的过程可以合在一起，以上是具体的细节实现
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()


# 使用LCEL将不同的组件拼凑成一个单独的链
chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})




"Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!"

“|” 符号- 类似于unix管道操作符，它将不同的组件链接在一起，将一个组件的输出作为输入提供给下一个组件。

# rag 案例


In [49]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [54]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

#cc：奇怪的问题，我已经 安装了，也更新了pip还是报这个错误。


ImportError: Could not import docarray python package. Please install it with `pip install "langchain[docarray]"`.